In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00397944450378418
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 10.482163789343398
200000 17.900767374891384
300000 19.522718107969148
400000 19.322833217962405
500000 19.503240614680177
600000 20.27554283723211
700000 19.233151481620787
800000 19.191405102550974
900000 19.408989706599936
1000000 17.06244757432685
1100000 9.038872071783477
1200000 19.056530713085344
1300000 19.078553867616044
1400000 19.000123544596875
1500000 19.2712340355724
1600000 20.11901098922535
1700000 18.936632054058137
1800000 19.000180974641697
1900000 19.161123392812126
2000000 19.620365747312853
2100000 5.134116221572029
2200000 21.510400411832826
2300000 23.219390448631305
2400000 22.429630121243243
2500000 22.841074840069915
2600000 23.20276944032703
2700000 21.81277037934687
2800000 21.38663402810959
2900000 21.649053394329265
3000000 21

29400000 17.062583499659947
29500000 17.366577393311086
29600000 16.66617393663055
29700000 11.715355609852553
29800000 11.642242144489025
29900000 16.62125527767714
30000000 17.256261380358083
30100000 17.083074176630433
30200000 18.42021924877578
30300000 18.002678526926225
30400000 18.46618686890927
30500000 12.076842130169904
30600000 14.662105403358053
30700000 18.42884536617074
30800000 18.348090305554706
30900000 17.989091913065664
31000000 9.639993985428925
31100000 15.953812407962126
31200000 18.22481307829534
31300000 17.93535696475675
31400000 17.747162479954905
31500000 16.92019744110862
31600000 16.209496711535394
31700000 16.81758279705245
31800000 5.4950174846050315
31900000 16.92617227599357
32000000 16.183511261117065
32100000 16.648582347028043
32200000 16.859445260767824
32300000 17.982533862917023
32400000 17.83739116258638
32500000 16.093781249639665
32600000 10.562667231927106
32700000 17.96664008323411
32800000 18.04689590952313
32900000 18.512998832899125
330000

59100000 15.628795179414723
59200000 16.110428986714354
59300000 15.97246442216633
59400000 15.800218865120215
59500000 16.526896695666743
59600000 15.877088401052989
59700000 16.28196225745706
59800000 15.494881857476074
59900000 8.710520935577327
60000000 13.114160393576181
60100000 14.082751304397217
60200000 14.63680096996534
60300000 14.060370621019981
60400000 12.890253140546314
60500000 6.919598339682875
60600000 14.051994514941498
60700000 14.179443798674885
60800000 13.79079490625192
60900000 13.90815440913569
61000000 8.560732971193136
61100000 14.100238887621762
61200000 15.574517441177365
61300000 15.479652234634068
61400000 16.212288621820317
61500000 16.520814198973866
61600000 16.249326167181515
61700000 15.399647933822411
61800000 15.632995105639223
61900000 15.850463829205292
62000000 6.439121761260381
62100000 14.25208500669315
62200000 14.03544015783823
62300000 13.86339967263901
62400000 14.020487803437241
62500000 6.840201811127246
62600000 13.221933579318822
62700

88700000 11.656508353570455
88800000 7.010785996887327
88900000 10.04823503442383
89000000 12.293990950415644
89100000 8.314005909581963
89200000 8.092892610009638
89300000 11.560261678159545
89400000 11.061012712478673
89500000 7.080607208520643
89600000 11.779618923879315
89700000 11.852451425313605
89800000 12.157394575709716
89900000 13.07065907310085
90000000 13.429199375916493
90100000 12.837095350321787
90200000 13.169572785049287
90300000 13.215995817256431
90400000 12.818060688635267
90500000 12.703441802066195
90600000 12.315370076895626
90700000 11.811544777120844
90800000 11.943719352044276
90900000 5.161553443878981
91000000 11.780950315019057
91100000 11.79530364475197
91200000 4.257409996153311
91300000 12.013107514439353
91400000 11.681219893528349
91500000 5.796893183580927
91600000 12.33293917418836
91700000 11.706606278619661
91800000 11.878012177022995
91900000 12.426121659250546
92000000 12.677227458933904
92100000 13.08098620213342
92200000 12.97986488766733
92300

117800000 3.292178104761197
117900000 4.158601358739429
118000000 8.834217128001642
118100000 9.67427479042122
118200000 9.690246816439213
118300000 9.689223581923724
118400000 9.970483869542885
118500000 9.72841097549713
118600000 10.106199483613318
118700000 10.366234496445928
118800000 10.926678143798027
118900000 10.95283510352057
119000000 9.769254983420788
119100000 9.868461797880899
119200000 9.297654086758115
119300000 9.630411527694136
119400000 9.526698640101474
119500000 9.610767866981602
119600000 9.343959721633647
119700000 9.695295855327014
119800000 3.176883489858399
119900000 1.764721315779241
120000000 7.023907497742039
120100000 9.349510802101953
120200000 9.334775958907628
120300000 9.557863303659069
120400000 9.463084044912266
120500000 9.79076528687284
120600000 9.33057019492125
120700000 9.918551362569733
120800000 9.808412749541045
120900000 11.5190460973662
121000000 10.958359466906886
121100000 10.502158132936382
121200000 10.268358857860985
121300000 9.8426322

146700000 6.385575508810119
146800000 7.650914046140704
146900000 7.91834177927227
147000000 8.112904686465752
147100000 7.967249967820009
147200000 8.1060251569327
147300000 7.9950571847773775
147400000 8.732380043556114
147500000 8.759746440021438
147600000 9.10307423332039
147700000 8.421661926523742
147800000 7.87763621567939
147900000 8.086384693260241
148000000 8.055987114741773
148100000 8.016330312915999
148200000 7.891560680452186
148300000 7.625483060540506
148400000 3.820688836134539
148500000 6.3348330342343555
148600000 5.28053598326581
148700000 3.4877767158547974
148800000 7.7476034015470345
148900000 7.855106777811988
149000000 8.019671290514832
149100000 7.908077997205127
149200000 8.057408151891751
149300000 7.821029253497791
149400000 8.379707057421957
149500000 8.7216992485614
149600000 8.474798718148522
149700000 8.648343028871299
149800000 7.904638832906756
149900000 7.810661051567012
150000000 7.8563522762004085
150100000 7.784561601176816
150200000 7.65008146912

176000000 6.375100332611025
176100000 6.87911227722595
176200000 6.3940287570295435
176300000 6.790945844709536
176400000 6.3706695566759395
176500000 6.674220032589355
176600000 6.070485127604496
176700000 6.026542777943932
176800000 4.837907074511852
176900000 3.881057727884817
177000000 6.224157762087311
177100000 5.906499488894271
177200000 3.1445544574330886
177300000 5.304730708638474
177400000 5.676709461523473
177500000 5.738197210234726
177600000 3.0182044377583077
177700000 5.7454168805285475
177800000 5.731288104083949
177900000 6.698219440663566
178000000 6.106659835120146
178100000 6.7977056989701055
178200000 6.601313713516521
178300000 6.694041353564308
178400000 6.158833048709406
178500000 6.512883235037473
178600000 6.102618129208823
178700000 6.124030111477744
178800000 5.608878860193665
178900000 2.697012390739187
179000000 6.134248498344638
179100000 5.629011495569412
179200000 5.383957024746887
179300000 2.833879370532197
179400000 5.44798757146182
179500000 5.7578

205200000 4.459834969372085
205300000 2.064715499323717
205400000 4.143747771174885
205500000 4.077841013072795
205600000 3.959685988679336
205700000 3.9689736274119496
205800000 3.867581932462096
205900000 1.4149181129797783
206000000 4.1007321851687
206100000 3.8788283606065623
206200000 3.840811820141765
206300000 3.95179185874908
206400000 4.115607915690221
206500000 1.759370067943922
206600000 4.383651986719823
206700000 4.366066865588618
206800000 4.3418914009832985
206900000 4.351428591509872
207000000 4.338689292087843
207100000 4.240962123264015
207200000 4.3011191141590235
207300000 2.9134198188019362
207400000 3.0754495194010993
207500000 3.9909440164909005
207600000 3.8697836940765638
207700000 3.6452250356330116
207800000 3.6677559819432517
207900000 2.902010052227213
208000000 2.3402000013428457
208100000 3.707524801888394
208200000 3.6511508290385364
208300000 3.7811377484903037
208400000 3.900014102505219
208500000 3.651525717192498
208600000 2.144472121133499
208700000

234000000 2.2856137925954796
234100000 2.141144196784606
234200000 2.191633142670343
234300000 2.1008133088421794
234400000 2.0748321721587812
234500000 2.062926520407075
234600000 0.7216894109181418
234700000 2.03318257200003
234800000 2.0584906446048934
234900000 2.090107783991506
235000000 2.119384202368074
235100000 2.0836907764472783
235200000 2.164105512817148
235300000 2.119172772829024
235400000 0.9276753632872581
235500000 2.1502788572406266
235600000 2.180854568491265
235700000 2.1011436492557127
235800000 0.9346285178796339
235900000 2.0904243818357036
236000000 2.0325602551736415
236100000 1.9975446668844565
236200000 1.879279690847412
236300000 1.9796660504588661
236400000 1.887244039724591
236500000 1.821974691994857
236600000 1.0882764967879037
236700000 1.511852095690218
236800000 1.8380097438144287
236900000 1.88734801984425
237000000 1.9974961822842279
237100000 1.949430307022261
237200000 1.9147402984006703
237300000 1.949426459058647
237400000 1.7952050695297164
237

262000000 0.001317036126622889
262100000 0.0006774267977158229
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 6.0087961953878406e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
